###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Instituto Nacional de Medicina Legal y Ciencias Forenses

In [ ]:
import pandas as pd
import numpy as np

###Archivo 8hqm-7fdt.csv

Contiene los datos históricos del Sistema de Información Red de Desaparecidos y Cadáveres. - SIRDEC de personas reportadas como desaparecidos desde el año 1930 hasta junio del año 2024.información delitos Sexuales desde el 01 de enero del año 2010 al 30 de abril del año 2024.

In [ ]:
# Nueva lista de nombres para las columnas del dataset
new_column_names = ['id', 'estado_actual', 'clasificacion', 'fecha_hecho',
                   'edad', 'sexo', 'pais_nacimiento', 'etnia', 'estado_civil',
                   'escolaridad', 'grupo', 'pais_del_hecho', 'municipio', 'departamento', 'contexto']

# Importando el CSV con los nuevos nombres de columna
df = pd.read_csv("https://www.datos.gov.co/resource/8hqm-7fdt.csv?$limit=200000", header=0)
df.columns = new_column_names

df.head()

,id,estado_actual,clasificacion,fecha_hecho,edad,sexo,pais_nacimiento,etnia,estado_civil,escolaridad,grupo,pais_del_hecho,municipio,departamento,contexto
0,1,Desaparecido,Sin Información,2006-12-29T00:00:00.000,21,Hombre,Sin información,Mestizo,Soltero (a),Educación básica secundaria o secundaria baja,Persona adicta a una droga natural o sintética,Colombia,Bogotá D.C.,Bogotá D.C.,Desaparición
1,2,Desaparecido,Sin Información,2006-12-26T00:00:00.000,24,Hombre,Colombia,Mestizo,Soltero (a),Educación básica secundaria o secundaria baja,Persona adicta a una droga natural o sintética,Colombia,Bogotá D.C.,Bogotá D.C.,Desaparición
2,3,Desaparecido,Sin Información,2006-12-29T00:00:00.000,13,Mujer,Colombia,Mestizo,Soltero (a),Educación básica secundaria o secundaria baja,Ninguno,Colombia,Bogotá D.C.,Bogotá D.C.,Desaparición
3,4,Aparecio Vivo,Sin Información,2006-12-29T00:00:00.000,17,Mujer,Sin información,Mestizo,Soltero (a),Educación técnica profesional y tecnológica,Ninguno,Colombia,Bogotá D.C.,Bogotá D.C.,Desaparición
4,5,Aparecio Vivo,Sin Información,2006-12-31T00:00:00.000,71,Hombre,Colombia,Mestizo,Unión Libre,Educación básica primaria,Otro,Colombia,Bogotá D.C.,Bogotá D.C.,Desaparición


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188218 entries, 0 to 188217
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               188218 non-null  int64 
 1   estado_actual    188218 non-null  object
 2   clasificacion    188218 non-null  object
 3   fecha_hecho      188218 non-null  object
 4   edad             188218 non-null  object
 5   sexo             188218 non-null  object
 6   pais_nacimiento  188218 non-null  object
 7   etnia            188218 non-null  object
 8   estado_civil     188218 non-null  object
 9   escolaridad      188218 non-null  object
 10  grupo            188218 non-null  object
 11  pais_del_hecho   188218 non-null  object
 12  municipio        188218 non-null  object
 13  departamento     188218 non-null  object
 14  contexto         188218 non-null  object
dtypes: int64(1), object(14)
memory usage: 21.5+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [ ]:
relevant_cols = ['estado_actual', 'fecha_hecho', 'edad', 'sexo', 'pais_del_hecho', 'municipio', 'departamento']
missing_people = df[relevant_cols]
missing_people.head()

,estado_actual,fecha_hecho,edad,sexo,pais_del_hecho,municipio,departamento
0,Desaparecido,2006-12-29T00:00:00.000,21,Hombre,Colombia,Bogotá D.C.,Bogotá D.C.
1,Desaparecido,2006-12-26T00:00:00.000,24,Hombre,Colombia,Bogotá D.C.,Bogotá D.C.
2,Desaparecido,2006-12-29T00:00:00.000,13,Mujer,Colombia,Bogotá D.C.,Bogotá D.C.
3,Aparecio Vivo,2006-12-29T00:00:00.000,17,Mujer,Colombia,Bogotá D.C.,Bogotá D.C.
4,Aparecio Vivo,2006-12-31T00:00:00.000,71,Hombre,Colombia,Bogotá D.C.,Bogotá D.C.


### Convertir columna fecha_hecho a tipo date

In [ ]:
# Hacer una copia explícita del DataFrame
missing_people_ = missing_people.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
missing_people_['fecha_hecho'] = pd.to_datetime(missing_people_['fecha_hecho'], errors='coerce')

# Extraer solo el año
missing_people_['fecha_hecho'] = missing_people_['fecha_hecho'].dt.year

### Verificar valores nulos

In [ ]:
missing_people_.isnull().sum()

,0
estado_actual,0
fecha_hecho,126
edad,0
sexo,0
pais_del_hecho,0
municipio,0
departamento,0


 - Solucionando nulos de fecha_hecho

In [ ]:
# Filtrar y mostrar los valores nulos
missing_people_[missing_people_['fecha_hecho'].isnull()].head()

,estado_actual,fecha_hecho,edad,sexo,pais_del_hecho,municipio,departamento
7,Desaparecido,NaN,35,Hombre,Colombia,Bogotá D.C.,Bogotá D.C.
76,Desaparecido,NaN,35,Hombre,Colombia,Bogotá D.C.,Bogotá D.C.
77,Desaparecido,NaN,15,Mujer,Colombia,Bogotá D.C.,Bogotá D.C.
78,Desaparecido,NaN,27,Hombre,Colombia,Bogotá D.C.,Bogotá D.C.
79,Aparecio Vivo,NaN,28,Hombre,Colombia,Bogotá D.C.,Bogotá D.C.



Dado que las fechas de los eventos tienden a ser consecutivas y no varían demasiado, optamos por la imputacion Forward-fill (imputación con el valor anterior)

In [ ]:
# Imputacion con Forward-fill
missing_people_['fecha_hecho'] = missing_people_['fecha_hecho'].ffill()

In [ ]:
# Verificar imputación
missing_people_['fecha_hecho'] = missing_people_['fecha_hecho'].astype(int)
missing_people_.isnull().sum()

,0
estado_actual,0
fecha_hecho,0
edad,0
sexo,0
pais_del_hecho,0
municipio,0
departamento,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['estado_actual', 'sexo', 'edad', 'departamento', 'municipio', 'pais_del_hecho']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(missing_people_[column].unique())
    print()


Categorías en la columna 'estado_actual':
['Desaparecido' 'Aparecio Vivo' 'Aparecio Muerto']

Categorías en la columna 'sexo':
['Hombre' 'Mujer']

Categorías en la columna 'edad':
['21' '24' '13' '17' '71' '20' '35' '15' '40' '25' '28' '14' '19' '27'
 '29' '47' '45' '36' '87' '44' '43' '56' '11' '31' '22' '63' '39' '48'
 '18' '16' '38' '70' '23' '41' '58' '50' '33' '42' '52' '12' '7' '26' '9'
 '72' '32' '46' '30' '34' '51' '8' '74' '53' '79' '5' '57' '10' '65' '78'
 '69' '84' '83' '54' '6' '64' '73' '55' '76' '82' '49' '68' '62' '37' '75'
 '77' '60' '80' '88' 'Menores de 1 año' '66' '2' '85' '94' '4' '81' '89'
 '86' '59' '3' '67' '61' 'Sin información' '1' '90' '96' '91' '100' '93'
 '97' '92' '98' '95' '99' '104' '110' '105' '101' '107']

Categorías en la columna 'departamento':
['Bogotá D.C.' 'Cundinamarca' 'Tolima' 'Meta' 'Bolívar' 'Santander'
 'Sin Información' 'Boyacá' 'Norte De Santander' 'Antioquia' 'Caquetá'
 'Quindio' 'Valle Del Cauca' 'Caldas' 'Cesar' 'Atlántico' 'Huila'
 'Cas

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    missing_people_[col] = missing_people_[col].apply(remove_accents_and_special_chars)

- Codificar la columna sexo

In [ ]:
#Codificar la columna 'sexo'
missing_people_['sexo'] = missing_people_['sexo'].map({'HOMBRE': 1, 'MUJER': 0})


  - Convertir 'edad' en un rango categórico, primero manejando valores inválidos

In [ ]:
# Reemplazamos 'Sin información' y otros valores no numéricos por NaN
missing_people_['edad'] = pd.to_numeric(missing_people_['edad'], errors='coerce')

# Definir los rangos de edad y las etiquetas
bins = [0, 12, 18, 35, 60, np.inf]
labels = ['MENORES', 'ADOLESCENTES', 'ADULTOS_JOVENES', 'ADULTOS', 'ADULTOS_MAYORES']

# Crear la nueva columna 'grupo_edad' según los rangos definidos
missing_people_['edad'] = pd.cut(missing_people_['edad'], bins=bins, labels=labels, right=False)

# Verificar las primeras filas
missing_people_.head()

,estado_actual,fecha_hecho,edad,sexo,pais_del_hecho,municipio,departamento
0,DESAPARECIDO,2006,ADULTOS_JOVENES,1,COLOMBIA,BOGOTA D.C.,BOGOTA D.C.
1,DESAPARECIDO,2006,ADULTOS_JOVENES,1,COLOMBIA,BOGOTA D.C.,BOGOTA D.C.
2,DESAPARECIDO,2006,ADOLESCENTES,0,COLOMBIA,BOGOTA D.C.,BOGOTA D.C.
3,APARECIO VIVO,2006,ADOLESCENTES,0,COLOMBIA,BOGOTA D.C.,BOGOTA D.C.
4,APARECIO VIVO,2006,ADULTOS_MAYORES,1,COLOMBIA,BOGOTA D.C.,BOGOTA D.C.


 - Dejar solamente los desaparecidos en Colombia

In [ ]:
# Contar ocurrencias de cada categoría en la columna pais_del_hecho
missing_people_['pais_del_hecho'].value_counts()

,count
pais_del_hecho,
COLOMBIA,187053
VENEZUELA,445
ECUADOR,245
MEXICO,131
PERU,65
PAIS DE LA DESAPARICION SIN INFORMACION,47
PANAMA,42
ESTADOS UNIDOS,30
CHILE,23


In [ ]:
# Conservar únicamente las filas donde la columna pais_del_hecho es 'COLOMBIA'
missing_people_ = missing_people_[missing_people_['pais_del_hecho'] == 'COLOMBIA']

In [ ]:
# Verificar
missing_people_['pais_del_hecho'].value_counts()

,count
pais_del_hecho,
COLOMBIA,187053


## Agregar código del municipio a dataframe missing_people para facilitar la georeferenciación

### -  Cargar los datos con códigos de departamentos y municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


### -  Asegurarse de que los nombres de los departamentos estén escritos de manera identica en cada dataframe

  - Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
# Aplicar funcion a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(missing_people_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en missing_people_", "Departamentos dept_mpios_codes")

Departamentos en missing_people_ que no están en Departamentos dept_mpios_codes:
{'SIN INFORMACION', 'SAN ANDRES Y PROVIDENCIA'}


  - Solucionar discrepancias en nombres de departamento

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES Y PROVIDENCIA': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'}

# Reemplazar los nombres incorrectos  el dataframe missing_people_
missing_people_['departamento'] = missing_people_['departamento'].replace(depto_mapping)

Nota: Por tratarse de un proyecto que enfatiza en la georreferencia no nos interesan datos de los que no se tenga el departamento donde se produjo el hecho, por lo tanto borramos los registros "SIN INFORMACION" en la columna departamento

In [ ]:
# Conservar únicamente las filas donde la columna pais_del_hecho es 'COLOMBIA'
missing_people_ = missing_people_[missing_people_['departamento'] != 'SIN INFORMACION']
missing_people_[missing_people_['departamento'] == 'SIN INFORMACION']

,estado_actual,fecha_hecho,edad,sexo,pais_del_hecho,municipio,departamento


 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas de departamento
compare_lists(missing_people_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en missing_people_", "Departamentos dept_mpios_codes")

Departamentos en missing_people_ que no están en Departamentos dept_mpios_codes:
set()


  b) Aplicar coincidencia difusa para combinacion departamento + municipio en los dos dataframe

Realizar coincidencia difusa (fuzzy matching), significa que puede comparar dos cadenas de texto (strings) y medir su similitud, incluso si no son exactamente iguales. Esta técnica es muy útil en este caso porque si bien los nombres de los departamentos estan debidamente ajustados en los dos df,  los nombres de los municipios pueden tener diferencias (errores de tipeo, variantes en nombres, etc.).

El utilizar la columna del nombre del departamento, es importante en este caso porque en colombia existen municipios con el mismo nombre.


 - Instalar la thefuzz en el entorno de google colab, para realizar coincidencia difusa

In [ ]:
!pip install thefuzz

- Aplicar thefuzz para coincidencia difusa

In [ ]:
# Crear una clave única de departamento + municipio en ambos datasets
missing_people_['dept_mpio'] = missing_people_['departamento'] + '_' + missing_people_['municipio']
dept_mpios_codes['dept_mpio'] = dept_mpios_codes['dept_name'] + '_' + dept_mpios_codes['mupio_name']

In [ ]:
# Crear un diccionario de municipios y códigos basado en dept_mpios_codes
municipios_dict = dict(zip(dept_mpios_codes['dept_mpio'], dept_mpios_codes['dept_mpio_code']))

In [ ]:
from thefuzz import process

# Funcion para Usar fuzzy matching (thefuzz)
def get_best_match(row, municipios_dict, threshold=80):
    dept_mpio_sexual = row['departamento'] + '_' + row['municipio']

    # Buscar la mejor coincidencia en dept_mpios_codes usando fuzzy matching
    best_match, score = process.extractOne(dept_mpio_sexual, municipios_dict.keys())

    # Si la similitud supera el umbral definido, devolver el código mapeado
    if score >= threshold:
        return municipios_dict[best_match]
    else:
        return 'SIN DATOS'


In [ ]:
# Aplicar la función para encontrar la mejor coincidencia
missing_people_['codigo_dane'] = missing_people_.apply(
    lambda row: get_best_match(row, municipios_dict, threshold=80), axis=1)

  - Verificar resultados obtenidos

In [ ]:
missing_people_.head()

,estado_actual,fecha_hecho,edad,sexo,pais_del_hecho,municipio,departamento,dept_mpio,codigo_dane
0,DESAPARECIDO,2006,ADULTOS_JOVENES,1,COLOMBIA,BOGOTA D.C.,BOGOTA D.C.,BOGOTA D.C._BOGOTA D.C.,11001
1,DESAPARECIDO,2006,ADULTOS_JOVENES,1,COLOMBIA,BOGOTA D.C.,BOGOTA D.C.,BOGOTA D.C._BOGOTA D.C.,11001
2,DESAPARECIDO,2006,ADOLESCENTES,0,COLOMBIA,BOGOTA D.C.,BOGOTA D.C.,BOGOTA D.C._BOGOTA D.C.,11001
3,APARECIO VIVO,2006,ADOLESCENTES,0,COLOMBIA,BOGOTA D.C.,BOGOTA D.C.,BOGOTA D.C._BOGOTA D.C.,11001
4,APARECIO VIVO,2006,ADULTOS_MAYORES,1,COLOMBIA,BOGOTA D.C.,BOGOTA D.C.,BOGOTA D.C._BOGOTA D.C.,11001


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [ ]:
# Comparar listas de códigos
compare_lists(missing_people_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en missing_people_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en missing_people_ que no están en Códigos de municipios dept_mpios_codes:
{'SIN DATOS'}


In [ ]:
# Verificar cuantos municipios quedaron sin código
len(missing_people_[missing_people_['codigo_dane']== 'SIN DATOS'])

7

In [ ]:
# Verificar cuales municipios quedaron sin código
missing_people_[missing_people_['codigo_dane']== 'SIN DATOS']

,estado_actual,fecha_hecho,edad,sexo,pais_del_hecho,municipio,departamento,dept_mpio,codigo_dane
16346,DESAPARECIDO,2002,ADULTOS,1,COLOMBIA,MAPIRIPANA,GUAINIA,GUAINIA_MAPIRIPANA,SIN DATOS
23031,DESAPARECIDO,2004,ADULTOS_JOVENES,1,COLOMBIA,MAPIRIPANA,GUAINIA,GUAINIA_MAPIRIPANA,SIN DATOS
73589,DESAPARECIDO,2006,ADULTOS_JOVENES,1,COLOMBIA,MAPIRIPANA,GUAINIA,GUAINIA_MAPIRIPANA,SIN DATOS
151674,DESAPARECIDO,1997,ADOLESCENTES,0,COLOMBIA,MAPIRIPANA,GUAINIA,GUAINIA_MAPIRIPANA,SIN DATOS
155239,DESAPARECIDO,2002,ADULTOS_JOVENES,0,COLOMBIA,MAPIRIPANA,GUAINIA,GUAINIA_MAPIRIPANA,SIN DATOS
155673,DESAPARECIDO,1987,ADULTOS,1,COLOMBIA,MAPIRIPANA,GUAINIA,GUAINIA_MAPIRIPANA,SIN DATOS
159172,DESAPARECIDO,1998,ADOLESCENTES,1,COLOMBIA,MAPIRIPANA,GUAINIA,GUAINIA_MAPIRIPANA,SIN DATOS


In [ ]:
# Listado de municipios de Guania
dept_mpios_codes[dept_mpios_codes['dept_name'] == 'GUAINIA']

,dept_mpio_code,dept_name,mupio_name,dept_mpio
1092,94001,GUAINIA,INIRIDA,GUAINIA_INIRIDA
1093,94343,GUAINIA,BARRANCOMINAS,GUAINIA_BARRANCOMINAS
1094,94883,GUAINIA,SAN FELIPE,GUAINIA_SAN FELIPE
1095,94884,GUAINIA,PUERTO COLOMBIA,GUAINIA_PUERTO COLOMBIA
1096,94885,GUAINIA,LA GUADALUPE,GUAINIA_LA GUADALUPE
1097,94886,GUAINIA,CACAHUAL,GUAINIA_CACAHUAL
1098,94887,GUAINIA,PANA PANA,GUAINIA_PANA PANA
1099,94888,GUAINIA,MORICHAL,GUAINIA_MORICHAL


In [ ]:
# Como no hay ningun en Guania de nombre MAPIRIAPANA eliminamos esos 7 registros
missing_people_ = missing_people_[~(missing_people_['codigo_dane']== 'SIN DATOS')]

 - Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [ ]:
# Comparar listas de códigos
compare_lists(missing_people_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en missing_people_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en missing_people_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio', 'dept_mpio', 'pais_del_hecho']
final_missing_people = missing_people_.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_missing_people['source_id'] = 15

In [ ]:
final_missing_people.columns

Index(['estado_actual', 'fecha_hecho', 'edad', 'sexo', 'codigo_dane',
       'source_id'],
      dtype='object')

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'codigo_dane': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'sexo': 'sex',
    'edad': 'age_group',
    'estado_actual': 'current_status',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_missing_people.rename(columns=translation_map, inplace=True)

In [ ]:
final_missing_people.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186366 entries, 0 to 188217
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   current_status    186366 non-null  object  
 1   year_of_incident  186366 non-null  int64   
 2   age_group         185999 non-null  category
 3   sex               186366 non-null  int64   
 4   dane_code         186366 non-null  object  
 5   source_id         186366 non-null  int64   
dtypes: category(1), int64(3), object(2)
memory usage: 8.7+ MB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_missing_people.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/missing_people.csv', index=False)